In [1]:
!nvidia-smi

Tue May 30 15:01:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !rm -r "/content/New Plant Diseases Dataset(Augmented)/"

In [6]:
!unzip "/content/drive/MyDrive/Dataset/New Plant Diseases Dataset(Augmented).zip" -d "/content/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___healthy/508527ca-609b-427f-9b08-7ac3f3d45034___R.S_HL 8112 copy 2.jpg  
  inflating: /content/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___healthy/508527ca-609b-427f-9b08-7ac3f3d45034___R.S_HL 8112 copy 2_flipLR.jpg  
  inflating: /content/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___healthy/50d502de-24fe-4929-bd9d-9bbe9bdb6541___R.S_HL 8052 copy 2_flipLR.jpg  
  inflating: /content/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___healthy/50e026e1-cee6-4382-9664-5cd5be377e91___R.S_HL 8314 copy.jpg  
  inflating: /content/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___healthy/50f1396f-52da-4f95-84e0-aaa2a4fa9121___R.S_HL 0648 copy 3.jpg  
  inflating: /content/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___healthy/50f1396f-52da-4f95-84e0-aaa2a4fa9121___R.S_HL 0648 copy 3_flipLR.jpg  
  inflating:

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential 
from glob import glob

In [8]:
train_path = '/content/New Plant Diseases Dataset(Augmented)/train'
valid_path = '/content/New Plant Diseases Dataset(Augmented)/valid'

In [9]:
IMAGE_SIZE = [224, 224]

In [10]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 2s 0us/step


In [11]:
for layer in vgg.layers:
  layer.trainable = False

In [12]:
folders = glob('/content/New Plant Diseases Dataset(Augmented)/train/*')
x = Flatten()(vgg.output)

In [13]:
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [15]:
from keras.preprocessing.image import ImageDataGenerator 

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [17]:
training_set = train_datagen.flow_from_directory('/content/New Plant Diseases Dataset(Augmented)/train',
target_size = (224, 224),
batch_size = 64,
class_mode = 'categorical')

Found 7316 images belonging to 4 classes.


In [18]:
test_set = test_datagen.flow_from_directory('/content/New Plant Diseases Dataset(Augmented)/valid',
target_size = (224, 224),
batch_size = 64,
class_mode ='categorical') 

Found 1829 images belonging to 4 classes.


In [ ]:
r = model.fit_generator(
training_set,
validation_data=test_set,
epochs=15,
steps_per_epoch=len(training_set),
validation_steps=len(test_set) )

<ipython-input-19-0468884c042b>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/15
 93/115 [=======================>......] - ETA: 19s - loss: 0.3088 - accuracy: 0.8859

In [ ]:
# plot the accuracy
plt.plot(r.history['accuracy'],label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.ylabel('accuracy')
plt.xlabel('epoch') 
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# plot the loss 
plt.plot(r.history['loss'],label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend() 
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
from tensorflow.keras.models import load_model
model.save('model_VGG16.h5')